# Files

In [1]:
drive_path = 'c:/'
import numpy as np
import pandas as pd
import os
import sys
import matplotlib.pyplot as plt
from scipy.stats import ks_2samp
from scipy.stats import anderson_ksamp
from scipy.stats import kruskal
from scipy.stats import variation
%matplotlib
import seaborn as sns
from scipy.stats import zscore
from scipy.stats import nanmean
from scipy.stats import nanstd

Using matplotlib backend: Qt4Agg


In [2]:
filename='C:\Users\Annie\Documents\Data\Ca_Imaging\GoodFiles\\fullpeak.csv'
comp=pd.read_csv(filename)
comp_sorted=comp.reindex_axis(comp.mean().sort_values().index, axis=1)
comp_labels=pd.DataFrame(comp.Mouse)
comp_group=pd.DataFrame(comp.Group)
tmp=[comp_group,comp_labels,comp_sorted]
composite_full=pd.concat(tmp,axis=1)
cfull=pd.melt(composite_full,['Group','Mouse'],var_name="Odor")
# cfull['Mouse'] = cfull['Mouse'].apply(lambda x: x.split('_')[0])
# composite_full['Mouse']=composite_full['Mouse'].apply(lambda x:x.split('_')[0])

In [3]:
groupcolors={'Control':'r','Mint':'g','Hexanal':'b'};

In [6]:
len(cfull)

12945

# Count number of excitatory and inhibitory events per cell

In [7]:
# for x in composite_full.index:
EI=pd.DataFrame([])
for x in composite_full.index:
    a=0
    b=0
    for value in composite_full.iloc[x][2:]:
        if value>0:
            a=a+1
        elif value<0:
            b=b+1
    c=pd.DataFrame(composite_full.iloc[x][['Group','Mouse']]).T
    c=c.reset_index(drop=True)
    d=pd.concat([c,pd.DataFrame({'E':[a]}),pd.DataFrame({'I':[b]})],axis=1)
    EI=EI.append(d)

# GRAPH THE EXCITATION AND INHIBITION

In [228]:
sns.set(style="white")
sns.set_context('poster',font_scale=1.3)
plt.figure(figsize=(5,7))
# Initialize the matplotlib figure
# f, ax = plt.subplots(figsize=(3, 4))
sns.set_color_codes("pastel")
sns.barplot(x='Group', y="E", data=EI,
            label="E", palette=groupcolors);
sns.despine()

sns.set_color_codes("muted")
ax=sns.barplot(x="Group", y="I", data=EI,
            label="I", palette=groupcolors);

# Add a legend and informative axis label
# ax.legend(ncol=2, loc="upper right", frameon=True,fontsize=30)
# ax.set(xlim=(-1, 3), ylabel="Odor number",
#        xlabel="Group")
sns.despine(left=True, bottom=True)
plt.ylabel('Number of Odors');
plt.title('Response Type');
plt.xlabel('');
# plt.xticks(fontsize=30);
# plt.yticks(fontsize=30);
plt.tight_layout()

# Separate E/I dataframes

In [10]:
#Make separate excitatory and inhibitory dataframes
mindf=pd.DataFrame([])
maxdf=pd.DataFrame([])
zdf=pd.DataFrame([])
for x in cfull.index:
    if cfull.value[x]>0:
        a=pd.DataFrame(cfull.iloc[x]).T
        maxdf=maxdf.append(a,ignore_index=True)
    elif cfull.value[x]<0:
        a=pd.DataFrame(cfull.iloc[x]).T
        mindf=mindf.append(a,ignore_index=True)
    else:
        a=pd.DataFrame(cfull.iloc[x]).T
        zdf=zdf.append(a,ignore_index=True)

In [6]:
#Plot Excitatory by group
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("talk", font_scale=1.8);
plt.figure(figsize=(45, 20));
sns.boxplot(x='Odor',y='value',hue='Group',palette=groupcolors,data=maxdf[['Group','Odor','value']]);
sns.despine()
plt.ylabel('Integral', fontsize=48);
plt.title('Excitatory responses', fontsize=55);
plt.xlabel('Odor', fontsize=48);

In [14]:
#Plot Inhibitory by group
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("talk", font_scale=1.8);
plt.figure(figsize=(45, 20));
sns.boxplot(x='Odor',y='value',hue='Group',palette=groupcolors,data=mindf[['Group','Odor','value']]);
sns.despine()
plt.ylabel('Integral', fontsize=48);
plt.title('Inhibitory responses', fontsize=55);
plt.xlabel('Odor', fontsize=48);

# GRAPH ALL RESPONSES BUT SEPARATED

In [36]:
#All responses
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("poster", font_scale=1);
# plt.figure(figsize=(45, 20));
sns.boxplot(x='Odor',y='value',hue='Group',palette=groupcolors,data=mindf[['Group','Odor','value']]);
ax=sns.boxplot(x='Odor',y='value',hue='Group',palette=groupcolors,data=maxdf[['Group','Odor','value']]);
ax.legend_.remove()
sns.despine()
plt.ylabel('Peak', fontsize=34);
plt.title('All responses', fontsize=50);
plt.xlabel('Odor', fontsize=34);

In [12]:
one_mindf=mindf[['Group','Odor','value']][mindf.Odor.isin(['MS01','Hexanal01','IAA01','THA','AP','PA','Hexanone','EB'])]
conc_mindf=mindf[['Group','Odor','value']][mindf.Odor.isin(['MS01','MS05','MS10','Hexanal01','Hexanal05','Hexanal10','IAA01','IAA05','IAA10'])]
conc_mindf=conc_mindf.sort_values(by='Odor')

one_maxdf=maxdf[['Group','Odor','value']][maxdf.Odor.isin(['MS01','Hexanal01','IAA01','THA','AP','PA','Hexanone','EB'])]
conc_maxdf=maxdf[['Group','Odor','value']][maxdf.Odor.isin(['MS01','MS05','MS10','Hexanal01','Hexanal05','Hexanal10','IAA01','IAA05','IAA10'])]

one_mindf=one_mindf.replace(to_replace='MS01',value='MS')
one_mindf=one_mindf.replace(to_replace='Hexanal01',value='Hexanal')
one_mindf=one_mindf.replace(to_replace='IAA01',value='IAA')

one_maxdf=one_maxdf.replace(to_replace='MS01',value='MS')
one_maxdf=one_maxdf.replace(to_replace='Hexanal01',value='Hexanal')
one_maxdf=one_maxdf.replace(to_replace='IAA01',value='IAA')

sorter=('MS01','MS05','MS10','IAA01','IAA05','IAA10','Hexanal01','Hexanal05','Hexanal10')
conc_maxdf.Odor=conc_maxdf.Odor.astype('category')
conc_maxdf.Odor.cat.set_categories(sorter,inplace=True)
conc_maxdf=conc_maxdf.sort_values(['Odor'])

conc_mindf.Odor=conc_mindf.Odor.astype('category')
conc_mindf.Odor.cat.set_categories(sorter,inplace=True)
conc_mindf=conc_mindf.sort_values(['Odor'])

conc_mindf=conc_mindf.replace(to_replace='MS01',value='MS 1%')
conc_mindf=conc_mindf.replace(to_replace='MS05',value='MS 5%')
conc_mindf=conc_mindf.replace(to_replace='MS10',value='MS 10%')
conc_mindf=conc_mindf.replace(to_replace='IAA01',value='IAA 1%')
conc_mindf=conc_mindf.replace(to_replace='IAA05',value='IAA 5%')
conc_mindf=conc_mindf.replace(to_replace='IAA10',value='IAA 10%')
conc_mindf=conc_mindf.replace(to_replace='Hexanal01',value='Hexanal 1%')
conc_mindf=conc_mindf.replace(to_replace='Hexanal05',value='Hexanal 5%')
conc_mindf=conc_mindf.replace(to_replace='Hexanal10',value='Hexanal 10%')

conc_maxdf=conc_maxdf.replace(to_replace='MS01',value='MS 1%')
conc_maxdf=conc_maxdf.replace(to_replace='MS05',value='MS 5%')
conc_maxdf=conc_maxdf.replace(to_replace='MS10',value='MS 10%')
conc_maxdf=conc_maxdf.replace(to_replace='IAA01',value='IAA 1%')
conc_maxdf=conc_maxdf.replace(to_replace='IAA05',value='IAA 5%')
conc_maxdf=conc_maxdf.replace(to_replace='IAA10',value='IAA 10%')
conc_maxdf=conc_maxdf.replace(to_replace='Hexanal01',value='Hexanal 1%')
conc_maxdf=conc_maxdf.replace(to_replace='Hexanal05',value='Hexanal 5%')
conc_maxdf=conc_maxdf.replace(to_replace='Hexanal10',value='Hexanal 10%')

In [13]:
cat=('Control','Mint','Hexanal')

In [17]:
#1% CONCENTRATION
cat=('Control','Mint','Hexanal')
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("poster", font_scale=1.3);
plt.figure(figsize=(13,10));
ax=sns.boxplot(x='Odor',y='value',hue_order=cat,hue='Group',palette=groupcolors,data=one_maxdf);
f=sns.boxplot(x='Odor',y='value',hue_order=cat,hue='Group',palette=groupcolors,data=one_mindf);
# ax.legend_.remove()
sns.despine()
plt.ylabel('DF/F', fontsize=34);
plt.title('Peak DF/F, 1%', fontsize=40);
plt.xlabel('Odor', fontsize=34);
plt.ylim(-1,5)
plt.tight_layout()

In [15]:
#Multiple Concentration
cat=('Control','Mint','Hexanal')
sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("poster", font_scale=1.3);
plt.figure(figsize=(17,10));
sns.boxplot(x='Odor',y='value',hue_order=cat,hue='Group',palette=groupcolors,data=conc_mindf);
ax=sns.boxplot(x='Odor',y='value',hue_order=cat,hue='Group',palette=groupcolors,data=conc_maxdf);
# ax.legend_.remove()
sns.despine()
plt.ylabel('DF/F', fontsize=34);
plt.title('Peak DF/F, Multiple Concentrations', fontsize=40);
plt.xlabel('Odor', fontsize=34);
plt.ylim(-1,5)
plt.tight_layout()

# Median

In [15]:
#MEDIANS
medians=pd.DataFrame([])
df=conc_maxdf
for x in df.Group.unique():
    tmp=pd.DataFrame([])
    for y in df.Odor.unique():
        a=df[(df.Group==x)&(df.Odor==y)]['value'].dropna().median()
        m=pd.DataFrame({y:[a]})
        tmp=pd.concat([tmp,m],axis=1)
    b=pd.DataFrame({'Group':[x]})
#     c=pd.DataFrame({'Group':[df[df.Mouse==x]['Group'].iloc[0]]})
#     c=c.reset_index(drop=True)
    tmp=pd.concat([b,tmp],axis=1)
    medians=medians.append(tmp)
maxmedianmelt=pd.melt(medians,"Group",var_name="Odor")

# medians=pd.DataFrame([])
# df=conc_mindf
# for x in df.Group.unique():
#     tmp=pd.DataFrame([])
#     for y in df.Odor.unique():
#         a=df[(df.Group==x)&(df.Odor==y)]['value'].dropna().median()
#         m=pd.DataFrame({y:[a]})
#         tmp=pd.concat([tmp,m],axis=1)
#     b=pd.DataFrame({'Group':[x]})
# #     c=pd.DataFrame({'Group':[df[df.Mouse==x]['Group'].iloc[0]]})
# #     c=c.reset_index(drop=True)
#     tmp=pd.concat([b,tmp],axis=1)
#     medians=medians.append(tmp)
# minmedianmelt=pd.melt(medians,"Group",var_name="Odor")

#Plot everything by group
# sns.set(style="white", palette="muted", color_codes=True);
# sns.set_context("poster", font_scale=1.3);
# plt.figure(figsize=(17, 5));
# ax=sns.pointplot(x='Odor',y='value',hue='Group',palette=groupcolors,data=maxmedianmelt)
# sns.pointplot(x='Odor',y='value',hue='Group',palette=groupcolors,data=minmedianmelt)
# ax.legend_.remove()
# sns.despine()
# plt.ylabel('DF/F');
# plt.title('Median DF/F');
# plt.xlabel('Odor');
# plt.ylim(-0.2,0.6)
# plt.tight_layout();

,Group,MS 1%,MS 5%,MS 10%,IAA 1%,IAA 5%,IAA 10%,Hexanal 1%,Hexanal 5%,Hexanal 10%
0,Control,0.131603,0.110242,0.133259,0.219825,0.163350,0.242727,0.261897,0.289709,0.260020
0,Hexanal,0.216249,0.276020,0.280561,0.345489,0.317836,0.427837,0.489682,0.530648,0.395452
0,Mint,0.183475,0.215221,0.240923,0.264259,0.267651,0.376057,0.353220,0.371466,0.378844


# Comparison of E and I

In [195]:
#EI ratio, of number
#1% Concentration
eibdf=pd.DataFrame([])
for x in one_maxdf.Group.unique():
    for y in one_maxdf.Odor.unique():
        a=float(len(one_mindf[(one_mindf.Group==x) & (one_mindf.Odor==y)]['value']))
        b=float(len(one_maxdf[(one_maxdf.Group==x) & (one_maxdf.Odor==y)]['value']))
        EIB=a/b
        eibdf=eibdf.append(pd.DataFrame({'Group':[x],'Odor':[y],'EIB':[EIB]}))

sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("poster", font_scale=1.3);
plt.figure(figsize=(13,5));
ax=sns.pointplot(x='Odor',y='EIB',hue='Group',palette=groupcolors,data=eibdf);
sns.despine()
plt.ylabel('Inhibition/Excitation Ratio');
plt.title('Inhibition/Excitation');
plt.xlabel('Odor');
ax.legend_.remove()
plt.tight_layout()

In [194]:
#EI ratio, of number
#Many Concentration
eibdf=pd.DataFrame([])
for x in conc_maxdf.Group.unique():
    for y in conc_maxdf.Odor.unique():
        a=float(len(conc_mindf[(conc_mindf.Group==x) & (conc_mindf.Odor==y)]['value']))
        b=float(len(conc_maxdf[(conc_maxdf.Group==x) & (conc_maxdf.Odor==y)]['value']))
        EIB=a/b
        eibdf=eibdf.append(pd.DataFrame({'Group':[x],'Odor':[y],'EIB':[EIB]}))

sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("poster", font_scale=1.3);
plt.figure(figsize=(17,5));
ax=sns.pointplot(x='Odor',y='EIB',hue='Group',palette=groupcolors,data=eibdf);
plt.ylim(0,0.35)
sns.despine()
plt.ylabel('Inhibition/Excitation Ratio');
plt.title('Inhibition/Excitation');
plt.xlabel('Odor');
ax.legend_.remove()
plt.tight_layout()

In [179]:
#EI balance (comparison of summed I/E)
eibdf=pd.DataFrame([])
for x in maxdf.Group.unique():
    for y in maxdf.Odor.unique():
        a=mindf[(mindf.Group==x) & (mindf.Odor==y)]['value'].sum()
        b=maxdf[(maxdf.Group==x) & (maxdf.Odor==y)]['value'].sum()
        EIB=abs(a/b)
        eibdf=eibdf.append(pd.DataFrame({'Group':[x],'Odor':[y],'EIB':[EIB]}))

sns.set(style="white", palette="muted", color_codes=True);
sns.set_context("talk", font_scale=1.3);
plt.figure(figsize=(6,8));
sns.pointplot(x='Odor',y='EIB',hue='Group',palette=groupcolors,data=eibdf);
sns.despine()
plt.ylabel('EIB', fontsize=50);
plt.title('Excitation-Inhibition Balance', fontsize=55);
plt.xlabel('Odor', fontsize=48);